In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
from skimage.draw import circle, line
from skimage.measure import regionprops
import tqdm
import matplotlib.image as mpimg
from scipy.spatial import distance as dist
import tqdm

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
image_list = []
for filename in sorted(glob.glob('/home/jovyan/DATA_MASTER_PROJECT/CNN/A549_t5_t15_fullsize/*.tiff'), key=natural_keys): 
    im=cv2.imread(filename)
    if np.count_nonzero(im == 255) < 1000:
        image_list.append(im)

In [4]:
def return_cord(i_0):    
    cx = []
    cy = []

    shifted = cv2.pyrMeanShiftFiltering(i_0, 10, 50)
    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=9, labels=thresh)
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thresh)



    for label in np.unique(labels):
        if label == 0:
            continue




        mask = np.zeros(gray.shape, dtype='uint8')
        mask[labels == label] = 255
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        ((x, y), r) = cv2.minEnclosingCircle(c)



        cx.append(int(x))
        cy.append(int(y))
        
        l1 = np.column_stack((cx,cy))
        

    return l1, cx, cy

In [5]:
dict_cord = {}
cor = []
x = []
y = []
for i in tqdm.tqdm(image_list):
    cor_0,x0,y0 = return_cord(i)
    cor.append(cor_0)
    x.append(x0)
    y.append(y0)
    
    data={'X':x, 'Y':y, 'Cor':cor}
    data = pd.DataFrame(data)

    
dict_cord = data
    

100%|██████████| 62/62 [00:40<00:00,  1.53it/s]


In [6]:
len(dict_cord)

62

In [7]:
coords = dict_cord.values.tolist()

# First Image t0 to t1

In [8]:
def eucl_dist_matrix_t1(cor_t, cor_t1):
    
    d = dist.cdist(cor_t, cor_t1)
    rows = d.min(axis=1).argsort()
    cols = d.argmin(axis=1)[rows]
    
    list_id_index = (list(zip(rows, cols)))
    list_id_index = sorted(list_id_index)
    
    
    
    
    return rows, cols, list_id_index ,d 

In [9]:
def find_duplicates_col_t1(rows, columns,list_id_index, distance_matrix):
    
    col = columns.tolist()
    duplicates = []
    for item in col:
        if col.count(item) > 1:
            duplicates.append(item)
    duplicate_columns = set(duplicates)
    
    d = []
    r = []
    c = []
    for ix, (a, b) in enumerate(zip(rows, columns)):
        for i in duplicate_columns: 
            if b == i:
                d.append(distance_matrix[a][b])
                r. append(a)
                c.append(b)
                
    w = (list(zip(r,c,d)))
    w.sort(key = lambda t: t[1])
    w
    
    temp = {}
    for row, col, di in w:
        if col not in temp: # we see this key for the first time
            temp[col] = (row, col, di)
        else:
            if temp[col][2] > di: # the new date is larger than the old one
                temp[col] = (row, col, di)

    result = temp.values()

    result = list(result)

    l2set = set(result)
    colums_to_remove = [x for x in w if x not in l2set] # cordinates that have to be removed
     
    rem_row, rem_col, rem_dit = zip(*colums_to_remove)
    rem_cor = (list(zip(rem_row,rem_col)))
    
    rem_cor = set(rem_cor)
    
    keep_cor = [x for x in list_id_index if x not in rem_cor]

    keep_cor.sort(key = lambda t: t[1])
    
    id_cell, index = zip(*keep_cor)

    
    return id_cell, index, duplicate_columns

In [10]:
def remove_duplicates_t1(x, y, index):
    
    large = max(index)
    lst = list(range(0,(large+1)))
    
    ix = set(index)
    
    missing_ix = [x for x in lst if x not in ix]
    
    centroid_t = (list(zip(x,y)))
    
    real_centroid = [i for j, i in enumerate(centroid_t) if j not in missing_ix]
    
    x_i, y_i = zip(*real_centroid)
    
    stacket_centroids = np.column_stack((x_i,y_i))
    
    
    
    return(x_i, y_i, stacket_centroids)

In [11]:
# def get_image_t0(x, y, image, number_image):
    
#     df = pd.DataFrame(list(zip(x, y)), 
#                columns =['row', 'col']) 


#     x = []
#     y = []

#     for i in range(len(df)):
#         x.append(df.iloc[i,0])
#         y.append(df.iloc[i,1])
        



#     for idx, i in enumerate(x):
    


#         cv2.circle(image, (x[idx], y[idx]), 3, (0, 255, 255), -1)
#         cv2.putText(image, "{0}".format(idx), (x[idx] - 20, y[idx] - 20),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
#     cv2.imwrite('img_try_fina_track_{0}.png'.format(number_image), image)

In [12]:
# def get_image(x, y, id_cell, image, number_image):
    
#     df2 = pd.DataFrame(list(zip(x, y, id_cell)), 
#                columns =['row', 'col', 'id_cell']) 


#     x = []
#     y = []
#     corrdinate = []
#     for i in range(len(df2)):
#         x.append(df2.iloc[i,0])
#         y.append(df2.iloc[i,1])
#         corrdinate.append(df2.iloc[i,2])



#     for idx, i in enumerate(x):



#         cv2.circle(image, (x[idx], y[idx]), 3, (0, 255, 255), -1)
#         cv2.putText(image, "{0}".format(corrdinate[idx]), (x[idx] - 20, y[idx] - 20),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

#     cv2.imwrite('img_try_fina_track_{0}.png'.format(number_image), image)

In [13]:
rows_1_2, cols_1_2, list_id_index_1_2 ,d_1_2 = eucl_dist_matrix_t1(coords[0][2], coords[1][2])

In [14]:
id_cell, index, duplicate_columns = find_duplicates_col_t1(rows_1_2, cols_1_2, list_id_index_1_2, d_1_2)

In [15]:
x_1, y_1, new_cor_1 = remove_duplicates_t1(coords[1][0],coords[1][1], index)

In [16]:
# get_image_t0(coords[0][0],coords[0][1], image_list[0], '0')
# get_image(x_1, y_1, id_cell, image_list[1], '1')

In [17]:
new_coor = {'Cor_1':new_cor_1}
new_id = {'ID_1':id_cell}

# From t1 to rest of data set



In [18]:
# IF Index duplicates cols > 0

def remove_dup_cols(row1, col1, dup_set2, x0, y0, id_cell,d, list_id_index1):

    dis = []
    r = []
    c = []

    for ix, (a, b) in enumerate(zip(row1, col1)):
        for i in dup_set2: 
            if b == i:
                dis.append(d[a][b])
                r. append(a)
                c.append(b)
                    
    w = (list(zip(r,c,dis)))
    w.sort(key = lambda t: t[1])
        
    temp = {}
    for row, col, dis in w:
        if col not in temp: # we see this key for the first time
            temp[col] = (row, col, dis)
        else:
            if temp[col][2] > dis: 
                temp[col] = (row, col, dis)

    result = temp.values()

    result = list(result)

        
    l2set = set(result)
    l3 = [x for x in w if x not in l2set]
        
    rem_row, rem_col, rem_dit = zip(*l3)
    rem_cor = (list(zip(rem_row,rem_col)))
        
    rem_cor = set(rem_cor)
    keep_cor = [x for x in list_id_index1 if x not in rem_cor]     # cordinates that have to be removed
    keep_cor.sort(key = lambda t: t[1])
        
        
    new_rowt1, new_colt1 = zip(*keep_cor)
    large = len(new_rowt1)
    large
    lst = list(range(0,(large+1)))
    ix = set(new_rowt1)

    m = [x for x in lst if x not in ix]

    centroid_t0 = (list(zip(x0,y0,id_cell)))

    real_centroid0 = [i for j, i in enumerate(centroid_t0) if j not in m]

    x_0, y_0, id_cell0= zip(*real_centroid0)

    correct_0 = np.column_stack((x_0,y_0)) 
        
    return correct_0, id_cell0

In [19]:
# IF Index unused cols > 0

def remove_unused_cols (idx_unused_col, x1, y1):
    
    miss_idx = idx_unused_col
    miss_idx
    centroid_t1 = (list(zip(x1, y1)))
    real_centroid1 = [i for j, i in enumerate(centroid_t1) if j not in miss_idx]
    
    return real_centroid1
        


In [20]:
def euc_matrix_2(data_t1, data_t2):
    
    rows1, cols1, matrix1, d1 = eucl_dist_matrix_t1(data_t1, data_t2 )
    row1, col1 = zip(*matrix1)

    correct_1= []

    for ix, (i,j) in enumerate(zip(row1, col1)):
        correct_1.insert(ix, data_t2[j])
            
    x_1, y_1 = zip(*correct_1)
    
    return x_1, y_1

In [21]:
def final_fun(dup_set2, idx_unused_col, d1, id_cell, img_now, num_img, list_id_index1, cor_now,
             new_cor_before):
    
    row1, col1         = zip(*list_id_index1)
    x_before, y_before = zip(*new_cor_before)
    x_now, y_now       = zip(*cor_now)

    if len(dup_set2) > 0 and len(idx_unused_col) > 0:

        correct_0, id_cell_new = remove_dup_cols(row1, col1, dup_set2, x_before, y_before, id_cell,d1,list_id_index1)

        real_centroid1      = remove_unused_cols (idx_unused_col, x_now, y_now)

        xt1, yt1 = euc_matrix_2(correct_0 ,real_centroid1)

        #get_image(xt1, yt1, id_cell_new, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell_new



    elif len(dup_set2) > 0 and len(idx_unused_col) == 0:

        correct_0, id_cell_new = remove_dup_cols(row1, col1, dup_set2, x_before, y_before, id_cell, list_id_index1)

        xt1, yt1 = euc_matrix_2(correct_0 , cor_now)

        #get_image(xt1, yt1, id_cell0, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell_new


    elif len(dup_set2)  == 0 and len(idx_unused_col) > 0:

        real_centroid1      = remove_unused_cols (idx_unused_col, x_now, y_now)

        xt1, yt1 = euc_matrix_2(new_cor_before , real_centroid1)

        #get_image(xt1, yt1, id_cell, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell


    else:

        xt1, yt1 = euc_matrix_2(new_cor_before , cor_now)

        #get_image(xt1, yt1, id_cell, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell


In [22]:
for ix in range(len(image_list)):

    row, col, list_id_index1 ,d1 = eucl_dist_matrix_t1(list(new_coor.values())[ix], coords[2+ix][2])

    row1, col1 = zip(*list_id_index1)


    # Index duplicates cols


    duplicates2 = []
    for item in col1:
        if col1.count(item) > 1:
            duplicates2.append(item)
    dup_set2 = set(duplicates2)


    # Index unused cols

    r = len(list(new_coor.values())[ix])
    unused_col = list(range(0, r))
    unused = set(col1)
    idx_unused_col = [x for x in unused_col if x not in unused]


    n_cord_new, id_cell_new = final_fun(dup_set2, idx_unused_col, d1, list(new_id.values())[ix], image_list[2+ix], '{}'.format(2+ix), list_id_index1, coords[2+ix][2], list(new_coor.values())[ix])
    
    new_coor['Cor_{}'.format(2+ix)] = n_cord_new
    new_id['ID_{}'.format(2+ix)] = id_cell_new
    

SyntaxError: invalid syntax (<ipython-input-22-eee72160b911>, line 28)

In [23]:
coor = []
id_c = []
len_idc = []
len_coor= []
for k, v in new_coor.items():
    coor.append(v)
    len_coor.append(len(v))

for p, c in new_id.items():
    len_idc.append(len(c))
    id_c.append(c)

In [24]:
list(zip(len_coor,len_idc))

[(22, 22)]